## Summarization - Tensor2Tensor

In [10]:
# using tensor datasets for first try
from tensor2tensor import problems
PROBLEM = 'summarize_cnn_dailymail32k'
TMP_DIR = '/sampledata' # Where data files from internet stored
DATA_DIR = '/sampledata' # Where pre-prcessed data is stored

# Init problem T2T object the generated training data
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR) 

In [11]:
from tensor2tensor import problems

In [9]:
problems.available()

['algorithmic_addition_binary40',
 'algorithmic_addition_decimal40',
 'algorithmic_cipher_shift200',
 'algorithmic_cipher_shift5',
 'algorithmic_cipher_vigenere200',
 'algorithmic_cipher_vigenere5',
 'algorithmic_identity_binary40',
 'algorithmic_identity_decimal40',
 'algorithmic_math_two_variables',
 'algorithmic_multiplication_binary40',
 'algorithmic_multiplication_decimal40',
 'algorithmic_reverse_binary40',
 'algorithmic_reverse_binary40_test',
 'algorithmic_reverse_decimal40',
 'algorithmic_reverse_nlplike32k',
 'algorithmic_reverse_nlplike8k',
 'algorithmic_shift_decimal40',
 'algorithmic_sort_problem',
 'audio_timit_characters_tune',
 'audio_timit_tokens8k_test',
 'audio_timit_tokens8k_tune',
 'babi_qa_concat_all_tasks_10k',
 'babi_qa_concat_all_tasks_1k',
 'babi_qa_concat_task10_10k',
 'babi_qa_concat_task10_1k',
 'babi_qa_concat_task11_10k',
 'babi_qa_concat_task11_1k',
 'babi_qa_concat_task12_10k',
 'babi_qa_concat_task12_1k',
 'babi_qa_concat_task13_10k',
 'babi_qa_concat_

In [12]:
import tensorflow as tf
from tensor2tensor import models
from tensor2tensor.models.transformer import Transformer
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment

In [13]:
PROBLEM = 'summarize_cnn_dailymail32k'
MODEL = 'transformer'
# HPARAMS = 'transformer_base'
HPARAMS = 'transformer_prepend'

TRAIN_DIR = '/model_files'
DATA_DIR = '/sampledata' # Where pre-prcessed data is stored

# Init Hparams object from T2T Problem
hparams = create_hparams(HPARAMS)

# Make Chngaes to Hparams
hparams.batch_size = 1024
hparams.learning_rate_warmup_steps = 10000 #45000
hparams.learning_rate = .4


In [14]:
import json
# Can see all Hparams with code below
print(json.loads(hparams.to_json()))

{'activation_dtype': 'float32', 'add_relative_to_values': False, 'multiproblem_schedule_threshold': 0.5, 'learning_rate_cosine_cycle_steps': 250000, 'multiproblem_schedule_max_examples': 10000000.0, 'attention_dropout_broadcast_dims': '', 'kernel_width': 1, 'optimizer_adafactor_memory_exponent': 0.8, 'optimizer_adafactor_decay_type': 'pow', 'moe_overhead_eval': 2.0, 'multiproblem_fixed_train_length': -1, 'modality': {}, 'hidden_size': 512, 'multiproblem_max_target_length': -1, 'attention_key_channels': 0, 'layer_postprocess_sequence': 'da', 'min_length_bucket': 8, 'symbol_dropout': 0.0, 'symbol_modality_skip_top': False, 'pad_batch': False, 'optimizer_adafactor_factored': True, 'optimizer_momentum_momentum': 0.9, 'initializer_gain': 1.0, 'optimizer_zero_grads': False, 'layer_preprocess_sequence': 'n', 'multiproblem_reweight_label_loss': False, 'layer_prepostprocess_dropout': 0.1, 'multiproblem_label_weight': 0.5, 'max_length': 0, 'sampling_temp': 1.0, 'learning_rate': 0.4, 'video_num_i

In [15]:
# Initi Run COnfig for Model Training
RUN_CONFIG = create_run_config(
      model_dir= TRAIN_DIR, # Location of where model file is store
      # More Params here in this fucntion for controling how noften to tave checkpoints and more. 
    model_name = MODEL
)
#     run_config = trainer_lib.create_run_config(
#         model_name="transformer",
#         model_dir=algorithmic.TinyAlgo.data_dir,
#         num_gpus=0,
#         use_tpu=False)

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']


/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


In [16]:
# Create Tensorflow Experiment Object
tensorflow_exp_fn = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=PROBLEM,
        data_dir=DATA_DIR, 
        train_steps= 10000, #400000, # Total number of train steps for all Epochs
        eval_steps=100 # Number of steps to perform for each evaluation
    )


INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': 1000, 'use_tpu': False, '_num_ps_replicas': 0, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f69942e50b8>, '_save_summary_steps': 100, '_evaluation_master': '', '_is_chief': True, '_num_worker_replicas': 0, 't2t_device_info': {'num_async_replicas': 1}, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_model_dir': '/model_files', '_train_distribute': None, '_environment': 'local', '_eval_distribute': None, '_tf_random_seed': None, '_task_id': 0, '_device_fn': None, '_save_checkpoints_secs': None, '_task_type': None, '_protocol': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69942e50f0>, '_keep

In [ ]:
# tf.reset_default_graph()

In [17]:
# Kick off Training
tensorflow_exp_fn.train_and_evaluate()

INFO:tensorflow:Reading data files from /sampledata/summarize_cnn_dailymail32k-train*
INFO:tensorflow:partition: 0 num_data_files: 100
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32781_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32781_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32781_512.top
INFO:tensorflow:Base learning rate: 0.400000
INFO:tensorflow:Trainable Variables Total size: 60903936
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /model_files/model.ckpt-1000


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key training/transformer/symbol_modality_32781_512/shared/weights_0/Adam not found in checkpoint
	 [[node save/RestoreV2_1 (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/estimator.py:1403) ]]

Caused by op 'save/RestoreV2_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-

In [ ]:
# import re

In [ ]:
# testest = 'Additional option captured haha \n It is so funny 89 332 : more or less we can do it ! fun fun.'

# ttt = re.sub("[^a-z]+", " ", testest.strip().lower())

In [ ]:
# ttt

In [ ]:
from tensor2tensor.models import transformer
import tensorflow as tf

hparams = transformer.transformer_base()
encoder = transformer.TransformerEncoder(hparams, mode=tf.estimator.ModeKeys.TRAIN)
# x = <your inputs, which should be of shape [batch_size, timesteps, 1, hparams.hidden_dim]>
x = keras.backend.expand_dims(inputLayer, axis=2)
y = encoder({"inputs": x, "targets": 0, "target_space_id": 0})
y = keras.backend.squeeze(y[0], 2)
